# What are LangChain Chains

In LangChain, chains facilitate the creation of end-to-end RAG pipelines. They integrate various components into a user-friendly interface, including the model, prompt, memory, output parsing, and debugging capabilities. A chain does the following: 1) receives the user’s query as input, 2) processes the LLM’s response, and 3) returns the output to the user.

To design a custom pipeline, one can extend the Chain class. An example is the LLMChain, which represents the most basic chain type in LangChain and inherits from the Chain parent class.

## Generating Text with LLMChain

Several methods, each with a unique output format, are available for effectively using a chain. This section will create a bot to suggest contextually appropriate replacement words. The following code snippet uses the GPT-3 model via the OpenAI API. It employs the PromptTemplate feature from LangChain and unifies the process using the LLMChain class.

Set the OPENAI_API_KEY environment variable with your API credentials.

In [ ]:
import os
from langchain_custom_utils.helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

## Calling

In [ ]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a word to replace the following: {word}?"

# Set the "OPENAI_API_KEY" environment variable before running following line.
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("artificial")

The simplest use of the chain is the __call__ method. This method directly passes the input to the object during its initialization and returns the input variable and the model’s response, provided under the text key.

It is also possible to pass numerous inputs simultaneously and receive a list for each input using the `.apply()` method. The only distinction is that inputs are not included in the returned list but will be in the same order as the input.

In [ ]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]

llm_chain.apply(input_list)

The `.generate()` method provides a more detailed response by returning an instance of LLMResult. This instance includes additional information, such as the `finish_reason` key, which clarifies why the generation process concluded. It could indicate that the model chose to finish or exceeded the length limit. Other self-explanatory data includes the total number of used tokens and the model used.

In [ ]:
llm_chain.generate(input_list)

Another method to consider is `.predict()`, which can be used interchangeably with .`run()`. This method is particularly effective when dealing with multiple inputs for a single prompt, though it can also be utilized with a single input. The following prompt will give both the word to be substituted and the context that the model must examine:

In [ ]:
prompt_template = "Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="object")

The model effectively recommended “Ventilator” as an appropriate replacement for the word “fan” in the context of “objects.” Additionally, when the experiment is conducted with a different context, “humans”, the suggested replacement changes to “_Admirer_”. This demonstrates the model’s ability to adapt its responses based on the specified context.

In [ ]:
llm_chain.predict(word="fan", context="humans")

We can directly pass a prompt as a string to a Chain and initialize it using the `.from_string()` function as follows: `LLMChain.from_string(llm=llm, template=template)`:

In [ ]:
template = """Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"""
llm_chain = LLMChain.from_string(llm=llm, template=template)

## Parsers

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=[], output_parser=output_parser))

llm_chain.predict()

In [ ]:
llm_chain.predict_and_parse()

## Adding Memory with ConversationalChain

Depending on the application, memory is a component that enriches a chain. Using the `ConversationalBufferMemory` class, LangChain provides a `ConversationalChain` to track past queries and responses.

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory())

conversation.predict_and_parse(input="Answer briefly. write the first 3 options.")

When we ask it to return the following four replacement words, it uses the memory to keep the context of the current task.

In [ ]:
conversation.predict_and_parse(input="And the next 4?")

## Concatenating Chains with SequentialChain

Another helpful feature is using a sequential chain that concatenates multiple chains into one:

In [ ]:
# from langchain.chains import SimpleSequentialChain
# overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

The `SimpleSequentialChain` will start running each chain from the first index and pass its response to the next one in the list.

## Debugging Chains

Debugging LangChain chains helps identify errors, optimize performance, and ensure smooth interactions between components, improving the reliability of complex language model workflows.

Setting the `verbose` option to `True` allows you to see the inner workings of any chain. As shown in the code below, the chain will return the initial prompt and the output. The application determines the output. If there are more steps, it may provide more information.

In [ ]:
conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict_and_parse(input="Answer briefly. write the first 3 options.")

## Custom Chain

LangChain offers a range of predefined chains tailored for specific tasks, including the `TransformChain`, `LLMCheckerChain`, `LLMSummarizationCheckerChain`, and `OpenAPIEndpointChain`. These chains share common characteristics discussed earlier. Additionally, LangChain enables the creation of custom chains to meet unique requirements. This section focuses on constructing a custom chain to provide a word’s meaning and suggest an alternative.

The process begins by creating a new class that inherits its capabilities from the Chain class. To adapt this class to a specific task, it is necessary to implement three essential methods: the `input_keys` and `output_keys` methods to inform the model of the expected inputs and outputs and the `_call` for executing each link in the chain and integrating their outputs into a coherent result.

The following example shows how to create a custom chain that concatenates the outputs of two separate chains and returns it.

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

Using the `LLMChain` class, we’ll declare each chain independently and use our custom chain `ConcatenateChain` to combine the results of `chain_1` and `chain_2`:

In [ ]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")